Посчитаем долю блокировщиков рекламы по источникам трафика за 12-18 февраля 2018. Не учитывайте строчки, в которых наличие блокировщиков не определено. В качестве метрики возьмем визиты. Измерение для блокировщиков рекламы ym:s:hasAdBlocker, источник трафика - ym:s:TrafficSource.

Требыется выяснить - какой источник имеет самую большую долю визитов с использованием блокировщиков рекламы?

внутренние переходы: исп - 21. не исп -  46. отношение - 31,3

In [1]:
dimensions_full = ['ym:s:date', 'ym:s:deviceCategory', 'ym:s:browser','ym:s:hasAdBlocker','ym:s:trafficSource']
metrics_full = ['ym:s:visits', 'ym:s:users', 'ym:s:pageviews']

dimensions = ['ym:s:hasAdBlocker','ym:s:trafficSource']
metrics = ['ym:s:visits']


In [2]:
import requests
from yaml import load
from pprint import pprint

f = open('config.yaml', 'r')
config = load(f)
token = config['token']
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = '2018-02-12'
endDate = '2018-02-18'
counter = '21075004'

In [3]:
def make_request(**kwargs):

    """Запрос к API Яндекс.Метрики. Возвращает JSON ответа"""

    return requests.get(API_URL, params = kwargs).json()

In [4]:
data = make_request(date1 = startDate, date2 = endDate, dimensions = ','.join(dimensions), metrics = ','.join(metrics), id = counter, oauth_token = token, accuracy = 1)
pprint(data)

{'data': [{'dimensions': [{'id': 'no', 'name': 'Блокировщик не используется'},
                          {'icon_id': '2',
                           'icon_type': 'traffic-source',
                           'id': 'organic',
                           'name': 'Переходы из поисковых систем'}],
           'metrics': [474.0]},
          {'dimensions': [{'id': 'no', 'name': 'Блокировщик не используется'},
                          {'icon_id': '0',
                           'icon_type': 'traffic-source',
                           'id': 'direct',
                           'name': 'Прямые заходы'}],
           'metrics': [213.0]},
          {'dimensions': [{'id': 'no', 'name': 'Блокировщик не используется'},
                          {'icon_id': '8',
                           'icon_type': 'traffic-source',
                           'id': 'social',
                           'name': 'Переходы из социальных сетей'}],
           'metrics': [84.0]},
          {'dimensions': [{'id': 'yes', 'na

In [5]:
def json_handler(data):

    """Преобразование ответа из JSON в список листов"""
    report = []
    for line in data['data']:
        dimensions_list = [x['name'] for x in line['dimensions']]
        metrics_list = line['metrics']
        report.append( dimensions_list + metrics_list )

    return report

In [6]:
stat_lists=json_handler(data)
print(stat_lists)

[['Блокировщик не используется', 'Переходы из поисковых систем', 474.0], ['Блокировщик не используется', 'Прямые заходы', 213.0], ['Блокировщик не используется', 'Переходы из социальных сетей', 84.0], ['Блокировщик используется', 'Переходы из поисковых систем', 51.0], ['Блокировщик не используется', 'Внутренние переходы', 46.0], ['Блокировщик используется', 'Внутренние переходы', 21.0], ['Блокировщик используется', 'Прямые заходы', 14.0], ['Блокировщик используется', 'Переходы из социальных сетей', 8.0], ['Блокировщик не используется', 'Переходы по ссылкам на сайтах', 1.0]]


In [7]:
json_handler(data)

[['Блокировщик не используется', 'Переходы из поисковых систем', 474.0],
 ['Блокировщик не используется', 'Прямые заходы', 213.0],
 ['Блокировщик не используется', 'Переходы из социальных сетей', 84.0],
 ['Блокировщик используется', 'Переходы из поисковых систем', 51.0],
 ['Блокировщик не используется', 'Внутренние переходы', 46.0],
 ['Блокировщик используется', 'Внутренние переходы', 21.0],
 ['Блокировщик используется', 'Прямые заходы', 14.0],
 ['Блокировщик используется', 'Переходы из социальных сетей', 8.0],
 ['Блокировщик не используется', 'Переходы по ссылкам на сайтах', 1.0]]

In [8]:
"""Можно переписать этот код с помощью метода setdefault. 
Этот метод проверяет есть ли в словаре указанный ключ 'google'. 
Если есть, то оставляет соответствующее значение ключа прежним. 
Если ключа не оказалось, то подставляет указанное нами значение (в примере это значение 0). 
Тем самым после применения метода setdefault можно смело использовать прибавление 1 к ключу 'google'.
Независимо от того был ли этот ключ в словаре раньше:"""

stat_dict = {}

for line in stat_lists:

    addblocker=line[0]
    source=line[1]
    visits=int(line[2])

    stat_dict.setdefault(addblocker, {})
    stat_dict[addblocker].setdefault(source, 0)
    stat_dict[addblocker][source] = visits
stat_dict

{'Блокировщик не используется': {'Переходы из поисковых систем': 474,
  'Прямые заходы': 213,
  'Переходы из социальных сетей': 84,
  'Внутренние переходы': 46,
  'Переходы по ссылкам на сайтах': 1},
 'Блокировщик используется': {'Переходы из поисковых систем': 51,
  'Внутренние переходы': 21,
  'Прямые заходы': 14,
  'Переходы из социальных сетей': 8}}

Далее можно перетащить словарь или даже лист листов в датафрейм и там обрабатывать.

In [9]:
import pandas as pd
df=pd.DataFrame(stat_dict)
df

,Блокировщик не используется,Блокировщик используется
Внутренние переходы,46,21.0
Переходы из поисковых систем,474,51.0
Переходы из социальных сетей,84,8.0
Переходы по ссылкам на сайтах,1,NaN
Прямые заходы,213,14.0


In [10]:
df['Отношение %'] = (df['Блокировщик используется']/(df['Блокировщик используется'] + df['Блокировщик не используется']))*100

In [11]:
df

,Блокировщик не используется,Блокировщик используется,Отношение %
Внутренние переходы,46,21.0,31.343284
Переходы из поисковых систем,474,51.0,9.714286
Переходы из социальных сетей,84,8.0,8.695652
Переходы по ссылкам на сайтах,1,NaN,NaN
Прямые заходы,213,14.0,6.167401


Итак ответ: "Внутренние переходы" - самая высокая доля визитов с блокировщиком рекламы 31.34%